3.3 For this assignment, We will create a clustermap

# Set-up

In [61]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
import warnings
warnings.filterwarnings('ignore')

#Import
fp = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
mybytes = fp.read()

#change coding
html_doc = mybytes.decode("utf8")
fp.close()

#create soup
soup = BeautifulSoup(html_doc, 'lxml')


In [65]:
# Using beautiful soup and Inspect element allowed to find the html section
My_table = soup.find('table',{'class':'wikitable sortable'})

#create a dataframe with information
df = pd.read_html(str(My_table))
#all the information is placed on first entry; so take it out
df= df[0]

#Setup Ind/\ex
headers= ['Postcode', 'Bourough', 'Neighborhood']
df.columns= headers

#drop inedex method
df= df.drop(df.index[0])
#df

#Dealing with Missing Values
df.replace("Not assigned", np.nan, inplace = True) #Convert existing data missing information to nan
df.dropna(subset=["Bourough"],inplace=True) # drop any rows with missing values in any cell. INPLACE impact dataframe if true
df.reset_index(drop=True, inplace=True) #reindex

# Inefficient beautiful soup method
#S1=soup.body.find('div', class_='mw-body-content',id='bodyContent')
#S2= S1.find('div', id='mw-content-text')
#S3= S2.find('div', class_='mw-parser-output')
#S4= S3.find('table')#, class_='mw-parser-output')
#S5= S4.tbody.tr #[''#.th#.text#.text



In [66]:
# Step 1: Create an array with postcodes, and an empty spot to fill

PostAr= df.Postcode.unique() #unique list of postcode
#Create an array of x with a second entry with 'a' to signify string
x=[]
for post in PostAr:
    x.append([post,'a'])   

#Step 2: Fill the x array with the neighborhood
    # Alg1: if rayindex= dataframe postcode and neighborhood is not in value
    # Alg2: THEN we add the neighborhood in entry and repeat
for i,square in enumerate(x):    
    for index, row in df.iterrows():
        if (df.loc[index][0])== (x[i][0]) and (str(df.loc[index][2]) in x[i][1]) == False:
            x[i][1]= x[i][1] + ', ' + str(df.loc[index][2])
        else:
            pass


#Step 3: Remove 'a,' from the entries        
for i,square in enumerate(x):
    x[i][1]=x[i][1][3:]

 

In [67]:
# Drop third column
df1 = df.iloc[0:,[0,1]]
# Drop the duplicates of new dataframe
df2 = df1.drop_duplicates()
df2.reset_index(drop=True, inplace=True) #reindex

#Add a new column filled with a
df2['Neighborhood'] = 'a'

#Fill the Neighborhood column with x list
for i,square in enumerate(x):    
    for index, row in df2.iterrows():
        if df2.loc[index][0]== (x[i][0]):
            df2.loc[index][2]= (x[i][1])
        else:
            pass

# Replace missing Neighborhood with Bourough        
df2.loc[df2['Neighborhood'] == 'nan']
df2['Neighborhood'].replace('nan', "Queen's Park", inplace= True)
df2.iloc[4][2] #Test it




"Queen's Park"

# Assignment 2: Table with Latitude and Longitude

In [68]:
#step 1: Read Geospatial Coordicates Date

df3 = pd.read_csv('Geospatial_Coordinates.csv')

#Set default column and values
df2['Latitude'] = 43.806
df2['Longitude'] = -79.194

#df2.loc[1][3]
#print(df2.iat[1,3])

# Create a loop replacing column values using geocoordinates; Notice the iat command is used to replace values in column
for jindex,square in df3.iterrows():    
    for index, row in df2.iterrows():
        if df2.loc[index][0]== str(df3.loc[jindex][0]): #this is the common key for both tables
            #print(type(df3.loc[jindex][1]))
            #print(df2.iat[index,3])
            df2.iat[index,3]= df3.loc[jindex][1] #iat is used for replacement of values
            df2.iat[index,4]= df3.loc[jindex][2]
        else:
            pass

# Final Table        
#df2
       

# Assignment 3: Toronto  Analysis

Load Packages and select Bouroughs that have Toronto

In [74]:
#import Programs
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library


#Find all unique Bouroughs
df2.Bourough.unique()
#Keep only the bourough that have Toronto in it
#print(type(neighborhoodsTor.dtypes))
neighborhoodsTor= df2[df2['Bourough'].str.contains("Toronto")]
neighborhoodsTor.reset_index(drop=True, inplace=True) #reindex


#neighborhoodsTor.head()
type(neighborhoodsTor)


pandas.core.frame.DataFrame

Create a map of Toronto; I will use different colors for each borough

Key: Blue= Downtown; Central= Red; East= Yellow; West= Green

In [71]:
# create map of Toronto using latitude and longitude values; A zoom of 12 was selected to make it easier to find
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

neighborhoodsTorD= df2[df2['Bourough'].str.contains("Downtown")]
neighborhoodsTorC= df2[df2['Bourough'].str.contains("Central")]
neighborhoodsTorW= df2[df2['Bourough'].str.contains("West")]
neighborhoodsTorE= df2[df2['Bourough'].str.contains("East Toronto")] #there is another East Category

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoodsTorD['Latitude'], neighborhoodsTorD['Longitude'], neighborhoodsTorD['Bourough'], neighborhoodsTorD['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoodsTorC['Latitude'], neighborhoodsTorC['Longitude'], neighborhoodsTorC['Bourough'], neighborhoodsTorC['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#ff6666',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)      

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoodsTorE['Latitude'], neighborhoodsTorE['Longitude'], neighborhoodsTorE['Bourough'], neighborhoodsTorE['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#ffff66',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
    
# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoodsTorW['Latitude'], neighborhoodsTorW['Longitude'], neighborhoodsTorW['Bourough'], neighborhoodsTorW['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#66ff66',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)      
    
map_toronto



From the data, it is clear there is cluster of postcodes, but where are they from? We will use the counts to determine the frequency of the respective bouroughs

In [60]:
#value counts into frame
#good variable 
Bourough_Counts = neighborhoodsTor['Bourough'].value_counts().to_frame()
Bourough_Counts.rename(columns={'Bourough': 'value_counts'}, inplace=True)
print(Bourough_Counts)


                  value_counts
Downtown Toronto            18
Central Toronto              9
West Toronto                 6
East Toronto                 5


We can infer downtown Toronto has the greatest density of people, hence there are many postcodes available.